In [1]:
%load_ext autoreload
%autoreload 2

# Construct simple model and train on polynomial (X,Y) dataset. 
## Test test extrapolates 

In [2]:
import plotly.express as px
import torch as pt
from torch import nn
import numpy as np

In [14]:
X = np.arange(-20,20, .01)
Y = .001 * X**2

print(f'X size: {len(X)}')
train_fraction = .8
index_of_split = int(len(X) * train_fraction)

X_train = X[:index_of_split]
X_test = X[index_of_split:]

Y_train = Y[:index_of_split]
Y_test = Y[index_of_split:]

X size: 4000


In [15]:
import pandas as pd
df = pd.DataFrame(dict(X=X, Y=Y))
df['dataset'] = 'train'

df.loc[lambda df_: df_.index >= index_of_split, 'dataset'] = 'test'

px.line(df, x='X', y='Y', color='dataset')

In [55]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_reli_stack = nn.Sequential(            
            nn.Linear(1, 64),
            nn.ReLU(),
            nn.Linear(64,1)
        )
        
    def forward(self, x):
        return self.linear_reli_stack(x)

model = NeuralNetwork().to('cpu')

In [56]:
loss_fn = nn.MSELoss()
optimizer = pt.optim.Adam(model.parameters(), lr=0.0001)

In [57]:
def np_1d_array_to_tensor(x: np.ndarray) -> pt.tensor:
    return pt.tensor(x.reshape(-1,1)).float()

X_train_tensor = np_1d_array_to_tensor(X_train)
Y_train_tensor = np_1d_array_to_tensor(Y_train)

X_test_tensor = np_1d_array_to_tensor(X_test)
Y_test_tensor = np_1d_array_to_tensor(Y_test)

In [58]:
def test(X, Y, model, loss_fn) -> float:
    with pt.no_grad():
        pred = model(X)
        return loss_fn(pred, Y).item()

In [59]:
def train(X: pt.tensor,
          Y: pt.tensor,
          
          X_cv: pt.tensor,
          Y_cv: pt.tensor,
          
          model,
          loss_fn,
          optimizer, 
          iterations: int):
    for i in range(iterations):
        pred = model(X)
        loss = loss_fn(pred, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 500 == 0:
            test_loss = test(X_cv, Y_cv, model, loss_fn)
            print(f'Train loss: {round(loss.item(), 4)}, test loss: {round(test_loss,4)}, iter: {i}')

In [60]:
train(X=X_train_tensor, 
      Y=Y_train_tensor, 
      X_cv=X_test_tensor,
      Y_cv=Y_test_tensor,
      model=model, 
      loss_fn=loss_fn,
      optimizer=optimizer,
      iterations=10_000)

Train loss: 8.6795, test loss: 0.0595, iter: 0
Train loss: 0.0335, test loss: 0.0006, iter: 500
Train loss: 0.0144, test loss: 0.003, iter: 1000
Train loss: 0.0058, test loss: 0.0106, iter: 1500
Train loss: 0.0025, test loss: 0.0156, iter: 2000
Train loss: 0.0012, test loss: 0.016, iter: 2500
Train loss: 0.0007, test loss: 0.0147, iter: 3000
Train loss: 0.0004, test loss: 0.0133, iter: 3500
Train loss: 0.0002, test loss: 0.0125, iter: 4000
Train loss: 0.0002, test loss: 0.0121, iter: 4500
Train loss: 0.0001, test loss: 0.0118, iter: 5000
Train loss: 0.0001, test loss: 0.0115, iter: 5500
Train loss: 0.0001, test loss: 0.0113, iter: 6000
Train loss: 0.0001, test loss: 0.011, iter: 6500
Train loss: 0.0001, test loss: 0.0108, iter: 7000
Train loss: 0.0, test loss: 0.0106, iter: 7500
Train loss: 0.0, test loss: 0.0102, iter: 8000
Train loss: 0.0, test loss: 0.0098, iter: 8500
Train loss: 0.0, test loss: 0.0094, iter: 9000
Train loss: 0.0, test loss: 0.0092, iter: 9500


In [61]:
Y_train_pred: list = model(X_train_tensor).tolist()
Y_test_pred: list = model(X_test_tensor).tolist()

df_predictions = df.copy()
df_predictions.loc[lambda df: df.dataset.eq('train'), 'Y'] = Y_train_pred
df_predictions.loc[lambda df: df.dataset.eq('train'), 'dataset'] = 'train_pred'

df_predictions.loc[lambda df: df.dataset.eq('test'), 'Y'] = Y_test_pred
df_predictions.loc[lambda df: df.dataset.eq('test'), 'dataset'] = 'test_pred'

In [62]:
px.line(df.append(df_predictions), x='X', y='Y', color='dataset')